In [1]:
import pandas as pd

df = pd.read_csv("KIC-SW8.csv")

df.head(20)

,center_name,center_lat,center_lon,station_name,distance
0,DMC 마스터원,37.604780,126.897321,응암역 6호선,1753.637832
1,DMC 마스터원,37.604780,126.897321,구산역 6호선,1901.238769
2,DMC시티워크,37.590426,126.886496,수색역 경의중앙선,1334.809090
3,DMC시티워크,37.590426,126.886496,디지털미디어시티역 공항철도,1750.123254
4,DMC시티워크,37.590426,126.886496,디지털미디어시티역 경의중앙선,1913.519122
5,GL메트로시티 한강,37.580093,126.857535,양천향교역 9호선,1914.649943
6,MBN STUDIO 지식산업센터,37.649817,126.897819,삼송역 3호선,424.610830
7,MBN STUDIO 지식산업센터,37.649817,126.897819,지축역 3호선,1427.164179
8,고양아크비즈,37.639975,126.875403,원흥역 3호선,1207.913449
9,광양프런티어밸리6차,37.638444,126.875005,원흥역 3호선,1370.641803


In [2]:
import pandas as pd

# 데이터 로드
df = pd.read_csv("KIC-SW8.csv")

# 1. 주변 지하철 개수 계산
df['지하철_개수'] = df.groupby('center_name')['center_name'].transform('count')

# 2. 접근성 점수 계산
# 점수 계산: max(0, 2000 - 거리) / 20
df['score'] = (2000 - df['distance']).clip(lower=0) / 20

# 가중치 계산: 1 / (거리 + 1)
df['weight'] = 1 / (df['distance'] + 1)

# 가중 평균 접근성 점수 계산
weighted_scores = df.copy()
weighted_scores['weighted_score'] = weighted_scores['score'] * weighted_scores['weight']

# 그룹화하여 가중 평균 점수 계산
grouped = weighted_scores.groupby('center_name').agg(
    total_weighted_score=('weighted_score', 'sum'),
    total_weight=('weight', 'sum')
)
grouped['가중_평균_접근성'] = grouped['total_weighted_score'] / grouped['total_weight']

# 3. 최종 데이터프레임 생성
# 'center_name' 기준으로 병합
df_final = df[['center_name', '지하철_개수']].drop_duplicates().merge(
    grouped[['가중_평균_접근성']], left_on='center_name', right_index=True
)

# 결과 확인
print(df_final)

# 결과 저장
df_final.to_csv("지하철_접근성_결과.csv", index=False, encoding="utf-8")

             center_name  지하철_개수  가중_평균_접근성
0               DMC 마스터원       2   8.777024
2                DMC시티워크       3  18.616615
5             GL메트로시티 한강       1   4.267503
6      MBN STUDIO 지식산업센터       2  67.260576
8                 고양아크비즈       1  39.604328
...                  ...     ...        ...
4962        본에스티스 지식산업센터       4  43.057665
4966   인천 중구 항동7가 지식산업센터       1   1.075139
4967             유니콘 101       1   2.600283
4968  아산 탕정일반산업단지 지식산업센터       2  11.962431
4970          와촌동 지식산업센터       2  85.703212

[979 rows x 3 columns]


In [3]:
df = pd.read_csv("Subway_Score.csv")

df['가중_평균_접근성'] = df['가중_평균_접근성'].round(2)

# 결과 확인
print(df)

# 수정된 데이터프레임 저장
df.to_csv("Subway_Score_수정.csv", index=False, encoding="utf-8")

            center_name  지하철_개수  가중_평균_접근성
0              DMC 마스터원       2       8.78
1               DMC시티워크       3      18.62
2            GL메트로시티 한강       1       4.27
3     MBN STUDIO 지식산업센터       2      67.26
4                고양아크비즈       1      39.60
..                  ...     ...        ...
974        본에스티스 지식산업센터       4      43.06
975   인천 중구 항동7가 지식산업센터       1       1.08
976             유니콘 101       1       2.60
977  아산 탕정일반산업단지 지식산업센터       2      11.96
978          와촌동 지식산업센터       2      85.70

[979 rows x 3 columns]
